In [136]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import csr_matrix
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

In [137]:
train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

In [138]:
#Remove Useless Column
train_data = train_data.drop(columns=['PassengerId','Name'])
test_data = test_data.drop(columns=['Name', 'PassengerId'])

In [139]:
#find empty cells
np.where(pd.isnull(train_data))

(array([   7,   10,   15, ..., 8675, 8684, 8687], dtype=int64),
 array([10,  8,  2, ..., 10,  0,  1], dtype=int64))

In [140]:
#fill empty cells with mode
for (columnName, columnData) in train_data.iteritems():
    replacer = columnData.mode()[0]
    train_data[columnName].fillna(replacer, inplace = True)
#fill empty cells with mode
for (columnName, columnData) in test_data.iteritems():
    replacer = columnData.mode()[0]
    test_data[columnName].fillna(replacer, inplace = True)

In [141]:
test_data

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0
2,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0
3,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0
4,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
4272,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0
4273,Earth,False,G/160/P,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0
4274,Mars,True,D/296/P,55 Cancri e,18.0,False,0.0,0.0,0.0,0.0,0.0
4275,Europa,False,D/297/P,TRAPPIST-1e,18.0,False,0.0,2680.0,0.0,0.0,523.0


In [142]:
train_X = train_data.drop(columns=['Transported']).values
train_Y = train_data['Transported'].values
test_X = test_data.values

In [143]:
enc = OneHotEncoder(handle_unknown='ignore')
c0 = enc.fit_transform(train_X[:,0].reshape(8693,1))
test_c0 = enc.transform(test_X[:,0].reshape(4277,1))
print(enc.categories_)
enc = OneHotEncoder(handle_unknown='ignore')
c2 = enc.fit_transform(train_X[:,2].reshape(8693,1))
test_c2 = enc.transform(test_X[:,0].reshape(4277,1))
print(enc.categories_)
enc = OneHotEncoder(handle_unknown='ignore')
c3 = enc.fit_transform(train_X[:,3].reshape(8693,1))
test_c3 = enc.transform(test_X[:,0].reshape(4277,1))
print(enc.categories_)

[array(['Earth', 'Europa', 'Mars'], dtype=object)]
[array(['A/0/P', 'A/0/S', 'A/1/S', ..., 'T/2/P', 'T/2/S', 'T/3/P'],
      dtype=object)]
[array(['55 Cancri e', 'PSO J318.5-22', 'TRAPPIST-1e'], dtype=object)]


In [144]:
train_X = np.delete(train_X, 0, 1)
train_X = np.delete(train_X, 1, 1)
train_X = np.delete(train_X, 1, 1)
test_X = np.delete(test_X, 0, 1)
test_X = np.delete(test_X, 1, 1)
test_X = np.delete(test_X, 1, 1)

In [145]:
train_X = 1*train_X
test_X = 1*test_X

In [146]:
train_X = np.concatenate((train_X, c0.toarray(), c3.toarray()), axis=1)
test_X = np.concatenate((test_X, test_c0.toarray(), test_c3.toarray()), axis=1)

In [147]:
normalizer = Normalizer().fit(train_X)

In [148]:
train_X = normalizer.transform(train_X)
test_X = normalizer.transform(test_X)

In [149]:
train_Y = 1*train_Y
train_Y

array([0, 1, 0, ..., 1, 0, 1])

In [150]:
clf = AdaBoostClassifier(n_estimators = 100, random_state=0)
clf.fit(train_X, train_Y)

AdaBoostClassifier(n_estimators=100, random_state=0)

In [151]:
clf.score(train_X, train_Y)

0.7981134245945013

In [152]:
accuracy_score(train_Y, clf.predict(train_X))

0.7981134245945013

In [156]:
result = clf.predict(test_X)

In [164]:
csv_result = pd.read_csv('./test.csv')
csv_result['Transported'] = result.astype(bool)
csv_result[['PassengerId', 'Transported']].to_csv('output.csv')